# Persistent Database

*For this project we will need to have a persistent database to store and access the data. I will be using a local mySQL database and access it using R*

You will need to use the mySQL command line client to create the database.

## Saving Data to Database

### Load Libraries

In [2]:
# load libraries
library(RMySQL)

Loading required package: DBI


### Load Data

In [1]:
# Data accessed from: https://open.canada.ca/data/en/dataset/6e438d59-880a-4f5e-827d-6c35243cf10a
# Data accessed on: 2019-07-12

border_df_000 <- read.csv('24100002.csv')

str(border_df_000)

'data.frame':	769590 obs. of  17 variables:
 $ ï..REF_DATE           : Factor w/ 567 levels "1972-01","1972-02",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ GEO                   : Factor w/ 142 levels "Abercorn, Quebec",..: 15 15 15 15 15 15 15 15 15 15 ...
 $ DGUID                 : Factor w/ 11 levels "","2016A000011124",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ Trip.characteristics  : Factor w/ 3 levels "Total Canadian vehicles returning",..: 3 2 2 2 2 2 2 2 1 1 ...
 $ Length.of.stay        : Factor w/ 4 levels "Length of stay, total",..: 1 1 1 1 1 3 2 4 1 1 ...
 $ Mode.of.transportation: Factor w/ 4 levels "Automobiles",..: 2 2 1 4 3 1 1 1 2 1 ...
 $ UOM                   : Factor w/ 1 level "Number": 1 1 1 1 1 1 1 1 1 1 ...
 $ UOM_ID                : int  223 223 223 223 223 223 223 223 223 223 ...
 $ SCALAR_FACTOR         : Factor w/ 1 level "units ": 1 1 1 1 1 1 1 1 1 1 ...
 $ SCALAR_ID             : int  0 0 0 0 0 0 0 0 0 0 ...
 $ VECTOR                : Factor w/ 2130 levels "v114064843","v114064859"

### Create a connection to your database

In [3]:
mydb = dbConnect(MySQL(), user= 'root'
                 , password = 'password'
                 , dbname='bordercrossings', host = 'localhost')

### List tables in the database

In [4]:
# list the tables in your database
dbListTables(mydb)

[1] "border_df_000"

### Remove Conflicting Tables

In [5]:
# drop tables; table (border_df_000)
dbRemoveTable(mydb,"border_df_000")

[1] TRUE

### Save R dataframe to database

In [6]:
# overwrite will create a new table; append will add rows to an existing
dbWriteTable(mydb, name='border_df_000', value=border_df_000 # value is database table
             , overwrite = TRUE, append = FALSE)

[1] TRUE

**Notes:**

If you get a permission error:
* log into MySQL terminal
* type **SET GLOBAL local_infile = true;**
* type **SHOW GLOBAL VARIABLES LIKE 'local_infile';** 


### Check that it saved properly

In [7]:
rs_save = dbSendQuery(mydb, "select * from border_df_000")
border_df_000_save = fetch(rs_save, n=-1)

str(border_df_000_save)

'data.frame':	769590 obs. of  18 variables:
 $ row_names             : chr  "1" "2" "3" "4" ...
 $ ï..REF_DATE           : chr  "1972-01" "1972-01" "1972-01" "1972-01" ...
 $ GEO                   : chr  "Canada" "Canada" "Canada" "Canada" ...
 $ DGUID                 : chr  "2016A000011124" "2016A000011124" "2016A000011124" "2016A000011124" ...
 $ Trip.characteristics  : chr  "Total vehicles entering Canada" "Total United States vehicles entering" "Total United States vehicles entering" "Total United States vehicles entering" ...
 $ Length.of.stay        : chr  "Length of stay, total" "Length of stay, total" "Length of stay, total" "Length of stay, total" ...
 $ Mode.of.transportation: chr  "Mode of transportation, total" "Mode of transportation, total" "Automobiles" "Trucks" ...
 $ UOM                   : chr  "Number" "Number" "Number" "Number" ...
 $ UOM_ID                : num  223 223 223 223 223 223 223 223 223 223 ...
 $ SCALAR_FACTOR         : chr  "units " "units " "units " "